In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
train_full_data = pd.read_csv('https://raw.githubusercontent.com/NHANGIOI/Machine_Learning/refs/heads/main/home-data-for-ml-course/train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/NHANGIOI/Machine_Learning/refs/heads/main/home-data-for-ml-course/test.csv')
x_full = train_full_data.copy().drop(columns = ['SalePrice'])
x_train,x_valid,y_train,y_valid = train_test_split(x_full,train_full_data['SalePrice'],
                                                  train_size = 0.8, test_size = 0.2,random_state = 0)

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

cate_cols = [col for col in x_train.columns if x_train[col].dtypes == 'object']
num_cols = [col for col in x_train.columns if x_train[col].dtypes in ['int','float']]

numberical_transformer = SimpleImputer(strategy = 'constant')

categorical_transformer = Pipeline(steps = [
    ('missing',SimpleImputer(strategy = 'most_frequent')),
    ('cate', OneHotEncoder(handle_unknown = 'ignore'))
])

preprocessor = ColumnTransformer(transformers = [
    ('num',numberical_transformer,num_cols),
    ('cate',categorical_transformer,cate_cols)
])

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

def run_model_tree(maxx : int,x_train,x_valid,y_train,y_valid):
    #model = RandomForestRegressor(n_estimators = 100,random_state = 0)
    model = DecisionTreeRegressor(max_leaf_nodes = maxx,random_state = 0)
    my_full_processor = Pipeline(steps = [
        ('preprocess',preprocessor),
        ('train_model',model)
    ])
    my_full_processor.fit(x_train,y_train)
    MAE = mean_absolute_error(y_valid,my_full_processor.predict(x_valid))
    return MAE,pd.DataFrame({
        'Id' : x_test['Id'],
        'SalePrice' : my_full_processor.predict(x_test)
    })
minn, output = run_model_tree(2,x_train,x_valid,y_train,y_valid)
for i in range(3,501):
    tres, tout = run_model_tree(i,x_train,x_valid,y_train,y_valid)
    if(tres < minn):
        minn = tres
        output = tout
output.to_csv('/content/submission.csv', index = False)